In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI

client = OpenAI(api_key="sk-proj-EhMwa5EECVtbpgtbTLroT3BlbkFJWTqh2jvIkszX28jlydCA")

In [ ]:
client.chat.completions.create(
messages=[{"role": "user", "content":"hello"}],
model="gpt-4o")

ChatCompletion(id='chatcmpl-9rKGgQIqSrZNzBN7AlyQ1sLiQN5Z4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1722496322, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_4e2b2da518', usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17))

In [ ]:
import openai
import csv
import json

# OpenAI API key
openai.api_key = 'sk-proj-EhMwa5EECVtbpgtbTLroT3BlbkFJWTqh2jvIkszX28jlydCA'

# File paths
input_csv_file = '/content/drive/MyDrive/WhatApp Chat/New chats/cleaned_data.csv'
output_json_file = 'categorized_conversations.json'
output_csv_file = 'categorized_conversations.csv'

# List of possible categories
query_categories = [
    "Opening Hours",
    "Address",
    "Menu",
    "Complaint",
    "Reservation",
    "Incomplete Query",
    "Place Order",
    "Partnerships and Collaborations",
    "Portion Size Inquiry",
    "Delivery Status",
    "Payment Issues"
]

support_response_categories = [
    "Good",
    "Bad",
    "No Answer"
]

# Read conversations from CSV
conversations = []
with open(input_csv_file, newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    header = next(csv_reader)
    for row in csv_reader:
        conversations.append({"id": row[0], "conversation": row[1]})  # Adjust indices as needed

categorized_conversations = []

for conv in conversations:
    conversation = conv['conversation']

    # Split the conversation into smaller chunks
    chunk_size = 2000  # Adjust this value based on the average length of your conversations
    conversation_chunks = [conversation[i:i+chunk_size] for i in range(0, len(conversation), chunk_size)]

    categories_json = {"query_categories": [], "support_response_categories": []}

    for chunk in conversation_chunks:
        prompt = {
            "instruction": "You will be provided with a part of a conversation and a list of possible query categories and support response categories. Please analyze this part of the conversation and select the most relevant categories for both the query and the support response. You can select multiple query categories if applicable. Return the output in JSON format as specified below.",
            "conversation": chunk,
            "query_categories": query_categories,
            "support_response_categories": support_response_categories,
            "output_format": {
                "query_categories": [],
                "support_response_categories": []
            }
        }

        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an assistant helping to categorize conversations."},
                {"role": "user", "content": json.dumps(prompt)}
            ],
            temperature=0,
            n=1,
            stop=None
            #json_mode=True
        )

        # Accumulate categories from each chunk
        chunk_categories = json.loads(response.choices[0].message.content)
        categories_json['query_categories'].extend(chunk_categories['query_categories'])
        categories_json['support_response_categories'].extend(chunk_categories['support_response_categories'])

    # Remove duplicates from accumulated categories
    categories_json['query_categories'] = list(set(categories_json['query_categories']))
    categories_json['support_response_categories'] = list(set(categories_json['support_response_categories']))

    categorized_conversations.append({
        "id": conv['id'],
        "conversation": conversation,
        "query_categories": categories_json['query_categories'],
        "support_response_categories": categories_json['support_response_categories']
    })

# Save to JSON file
with open(output_json_file, 'w') as jsonfile:
    json.dump(categorized_conversations, jsonfile, indent=4)

# Save to CSV file
with open(output_csv_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["ID", "Conversation", "Query Categories", "Support Response Categories"])
    for conv in categorized_conversations:
        csv_writer.writerow([
            conv["id"],
            conv["conversation"],
            ", ".join(conv["query_categories"]),
            ", ".join(conv["support_response_categories"])
        ])

print("Categorization complete and saved to JSON and CSV files.")

In [ ]:
import openai

# Replace 'your-api-key' with your actual OpenAI API key
openai.api_key = 'sk-proj-EhMwa5EECVtbpgtbTLroT3BlbkFJWTqh2jvIkszX28jlydCA'

def verify_bangla_conversation(messages):
    try:
        response = openai.chat.completions.create(
            model="gpt-4",  # Specify the model as GPT-4
            messages=messages,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return str(e)

# Example Bangla conversation as messages
bangla_messages = [
    {"role": "system", "content": "You are a helpful assistant that understands Bangla."},
    {"role": "user", "content": "Apnader location kuthay?"}
]

# Verify Bangla conversation understanding
result = verify_bangla_conversation(bangla_messages)
print("Response from GPT-4:", result)


Response from GPT-4: Ami ekta artificial intelligence, tai amar kono sthan nai. Ami jekono jaigay mora thakte pari jekhane internet connection ache.
